## Chapter2 The Graphics Rendering Pipeline
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;本章讲述实时图形学的核心部分, 叫做图形渲染管线, 通常简称为管线.管线的主要作用是根据给定的虚拟摄像机,三维物体,光源等生成或渲染出一个二维图像.因此, 渲染管线是实时渲染的基础工具.管线的流程如图2.1所示.![image.png](RTR.2.1.png)
### 2.1 架构
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;物理世界中,管线有不同的表现形式,从工厂组装线到快餐厅, 也同样应用于图像渲染.管线由几个阶段组成.每个阶段执行一个大任务的一部分.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\color{red}{管线的各个阶段是并行执行的,每个阶段依赖于上一个阶段的结果}$(The pipeline stages execute in parallel, with each stage dependent upon the result of the previous stage.)理想情况下,一个非流水线式的系统分为$n$个流水线阶段可以使整体的执行效率加速$n$倍.性能的提升也是使用管线技术的主要原因.例如,一批人可以加工大量的三明治--一个准备面包,另一个加肉,另一个加配料.每个人将自己加工完成的三明治传递给流水线上的下一个人并直接开始加工下一个三明治.如果每个人执行自己的任务要花费20秒,那么最高效率下,20秒就可以产出一个三明治,一分钟生产三个也是可能的.虽然管线各阶段并行执行,但它们会被其中最慢的一个阶段阻塞.例如,加工肉的阶段变得比较复杂,要花费30秒,那么现在最高的产出率就是一分钟两个三明治.对于这种特殊的生成流水线来说,加工肉的阶段就成为了瓶颈,因为它决定了整个产品的加工速度.当加工配料的阶段在等待加工肉的阶段时就处在了饥饿状态,对消费者来说同样如此.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这种类型的管线结构同样存在于实时计算机图形学中.实时渲染管线可以粗略分为四个阶段--应用层,几何体处理,光栅化和像素处理,如图2.2所示.这种架构是渲染管线引擎的核心,应用于实时计算机图形学中,因此也是接下来的章节所要讨论的内容的重要基础.每个阶段也自成一个管线,即由几个子阶段组成.我们在此把功能阶段和它们的实现结构分开.功能阶段会执行一个特定的任务单并不指定任务在管线中执行的方式.一个给定的实现可能会将两个功能阶段合并为一个单元或者将耗时长的功能阶段分配到几个硬件单元中用可编程核心来执行.![image.png](RTR.2.2.png)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;渲染速率可能会用frames per second(FPS)来表示,即每秒渲染的图像数量.也能用Hertz(Hz)表示,简记为$\frac{1}{seconds}$,也就是更新的速率.通常也用时间表示为毫秒(ms),也就是渲染一幅图像所耗费的时间.生成一幅图像的时间通常取决于每一帧的计算复杂度.每秒帧数不仅用来表示某一特定帧的速率,也表示某一阶段的平均效率.Hertz是硬件使用的参数,例如被设定到固定刷新频率的显示器.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;正如名称所表示的那样,应用层阶段被应用驱动因此通常由运行于普通cpu上的软件实现.这类cpu通常包括几个核心并且能并行运行多个线程.这让cpu能在应用层阶段高效运行大量任务.一些运行在cpu端的典型任务有碰撞检测,全局加速算法,动画,物理模拟等,取决于应用的类型.下一个主要的阶段是几何处理阶段,也就是处理变换,投影和所有其他类型的几何操作.这个阶段计算什么应该被显示以及如何被显示和在哪里显示.几何阶段通常在包含很多可编程核心和固定操作硬件的gpu(graphics processing unit)上执行.光栅化阶段通常将三个顶点作为输入,把它们格式化为一个三角形,并找出所有包含在三角形内的像素点,然后传输给下一个阶段.最终,像素处理阶段为每个像素执行一段程序来决定像素的颜色并有可能执行深度测试来决定这个像素是否可见.这个阶段也有可能执行一些逐像素操作,例如将新计算出的颜色与之前的颜色混合.光栅化和像素处理阶段都完全在gpu上执行,所有这些阶段和它们内部的管线都将在接下来的四个部分讨论.关于gpu如何处理这些阶段的更多细节将在第三章给出.
### 2.2 应用层阶段
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;应用层阶段通常在cpu上执行,所以开发者拥有完全的控制权.因此,开发者完全可以决定如何实现并在后续为了提高效率而修改实现.这个阶段的更改同样影响接下来的各个阶段的效率.例如,一个应用层阶段的算法或者配置,能减少需要渲染的三角形数量.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这就是说,某些应用层的工作可以由gpu通过一个叫compute shader的单独模式完成.这种模式下,gpu被当做一个高度并行的处理器,忽略它的特殊功能意味着它对渲染管线是特殊的.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;在应用层阶段的末尾,需要渲染的集合体被传输给几何处理阶段,这些是渲染图元,也就是点,线和三角形,即可在屏幕上显示的最终形式(或者任何其他输出设备).这是应用层阶段最重要的任务.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;一系列软件实现的应用层并没有被分成几个子阶段,几何处理阶段,光栅化和像素处理阶段同样如此.为了提高执行效率,应用层阶段通常在几个处理器核心上并行执行.这在cpu设计上叫做超标量体系架构,因为它能在同一阶段的同一时间执行几个过程.章节18.5展示了使用多个处理器核心的多种方式.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;一个通常实现于这个阶段的过程叫做碰撞检测.当检测到两个物体相碰撞时,可能会产生一个反馈并发回给碰撞物,也会发回给一个力反馈设备.应用层阶段也用来处理其他的设备输入,例如键盘,鼠标或者头戴式的显示器.基于这些输入可能会产生几种不同的行为.加速算法,特别是裁剪算法也实现于这个阶段,连同其它的所有管线无法处理的操作.
### 2.3 几何处理阶段
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;运行于gpu上的几何处理阶段负责大部分的逐三角形和逐顶点操作.这个阶段可细分为以下的功能阶段:顶点着色,投影,裁剪和屏幕映射.(如图2.3)![image.png](RTR.2.3.png)  
#### 2.3.1 顶点着色
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;顾名思义,顶点着色有两个主要任务,计算顶点的位置和计算顶点的输出数据,例如法线和纹理坐标.典型的为物体着色就是将光照信息应用到顶点的位置和法线,并只在顶点中存储最终颜色.然后在三角形中根据顶点颜色进行插值.随着现代gpu的出现,部分或全部这种类型的着色被逐像素着色所取代,这个顶点着色阶段或许完全不再执行着色方程,这取决于程序员的意愿.目前的顶点着色是一个用于配置每个顶点相关数据的更通用的单元.例如,vertex shader可以用来制作物体的动画.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;我们从描述顶点位置是如何计算的开始,一系列坐标系总是需要的.在显示到屏幕上的过程中,一个模型被变换到几个不同的空间和坐标系统中.开始时,模型位于模型空间中,也就是完全没有任何变换.每个模型都可以跟一个模型变换相联系以用于放置和转向.一个模型可能有几种模型变换.这就允许一个模型在同一个场景中拥有不同位置,转向和大小的多份实例,而不需要拷贝一个它的副本.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;模型变换变换的是顶点和模型的法线.一个物体的坐标系叫做模型坐标系,当模型变换应用到这些模型坐标系之后,就称模型被置于到了世界坐标系或者世界空间中.世界空间是唯一的,并且在所有的模型经过它们各自的模型变换之后都会处于同一个空间中.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;正如之前所提到的,只有摄像机或者观察者能看到的模型才会被渲染.摄像机被放置于世界空间中.为了投影和裁剪,摄像机和所有的模型都被应用了观察变换(view transform).观察变换的目的是把摄像机放置到原点并指向z轴的反方向,同时y轴指向上方,x轴指向右方.我们以z轴指向屏幕内为惯例,有些描述倾向于使z轴指向屏幕外.两者之间的区别大部分在于语义,因为这两者之间的转换很简单.观察变换后的实际位置和方向依赖于应用层的API.这个空间叫做摄像机空间或者更通用的叫法是观察空间或视觉空间.图2.4展示了观察变换是如何影响摄像机和模型的.模型变换和观察变换都可以实现为4x4矩阵.无论如何,重要的是要认识到顶点的位置和法线可以用程序员喜欢的任何方式来计算.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;接下来,我们要讲述顶点卓德第二种输出形式.为了产生一个真实的场景,不仅要渲染物体的形状和位置,也要对它们的表面进行建模.这就包括每个物体的材质,也包括物体上任何发光的光照效果,材质和灯光可以有多种表现形式,从简单的颜色到具体的物理描述.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这种决定灯光作用于材质上的效果的操作叫做着色.它涉及到物体上多个不同点上的着色方程.通常这些计算中的一部分是在几何处理阶段完成的,余下的则可能是在逐像素处理中执行.多种材质数据都可以在每个顶点中存储,例如顶点的位置,法线,颜色和一些其他的应用于着色方程的信息.顶点着色的结果被传输到光栅化和像素处理阶段用于插值和计算表面着色.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;作为顶点着色的一部分,渲染系统还要执行投影和裁剪,将观察体变换到(-1,-1,-1)到(1,1,1)的范围内.可以用不同的表示范围定义同一个观察体,例如(0<=z<=1>).单位立方体叫做规范观察体(cvv, canonical view volume).投影在gpu端被vertex shader首先完成.通常有两种投影方式,分别叫做正交(或平行)和透视投影.如图2.5.实际上,正交投影只是平行投影的一种.其他几种平行投影方式多在建筑学领域应用,例如斜投影和轴测投影.![image.png](RTR.2.5.png)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;注意投影被表示为一个矩阵,这样它就可以跟其余的几何变换相结合.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;正交投影的观察空间一般是一个长方体盒子,正交投影变换将观察体映射到单位立方体中.正交投影的主要特征是平行线在变换之后依然保持平行.这种变换是平移变换和缩放变换的结合.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;透视投影更复杂一些.这种投影钟,离摄像机越远的物体越小,平行线经变换后可能会在地平线处相交.因此.透视投影用来模拟我们观察物体的方式.从几何上讲,观察体叫做平截头体(frustum),像一个截去头部的金字塔.平截头体也会变换到单位立方体.正交变换和透视变换都可以构建为4x4矩阵,并且在两者中的任一变换后,我们就说模型被放置在了裁剪坐标系中.这些实际是齐次坐标系.所以这些发生在齐次除法之前.gpu的vertex shader必须总是输出这种类型的坐标系,为了保证正确的裁剪.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;尽管这些矩阵将一个观察体变换到另一个,它们叫做投影是因为在显示之后,z坐标不是存储在生成的图像中,而是存储在z-buffer中.这样,模型就从三维投影到了二维.
#### 2.3.2 可选的顶点处理
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;每个管线都包含上面描述的顶点处理过程.当这些处理完成后,还有一些可选的阶段可以在gpu端以以下顺序执行:曲面细分,几何着色和流输出.它们是否能被使用都依赖于硬件的能力--不是所有的gpu都有这些功能.它们是相互独立的.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第一个可选的阶段是曲面细分,想像你有一个球形物体,如果你用单个三角形集去表示它,你会遇到质量和效率问题.球形物体可能在5米外看着很不错,但当精细到单个三角形的时候,尤其是贴着轮廓线,问题就变得很明显.如果你用更多的三角形来提升质量,当球距离较远只覆盖了少量像素的时候就会浪费相当量的时间和内存.但用曲面细分,就可以生成含有适当数量三角形的曲面.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;我们已经讨论了一点三角形的问题,但目前为止我们在管线中只处理了顶点.这些顶点可以用来表示点,线,三角形或其他物体.顶点也能用来表示曲面,例如球.这类表面可以由一系列patches(???)指定,每个patch由一系列顶点集组成.曲面细分阶段本身也由几个阶段组成--hull shader, tessellator和domain shader--转换这些顶点集到用于生成三角形集的大顶点集中.场景中的摄像机可以用来决定要生成多少个三角形.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;下一个可选的阶段是geometry shader.这个shader跟曲面细分的shader有些相似,它以不同类型的图元作为输入来产生新的顶点.但它限定于特定的阶段并且对于输出的图元类型也有限制.geometry shader有多种用途,其中最常用的一种是生成粒子.设想模拟烟火爆炸,每一个火球都可以用一个顶点表示.geometry shader可以把这些顶点转换成朝向观察者且覆盖了一些像素点的方形(两个三角形组成),以提供一个更合适的图元来着色.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;最后一个可选的阶段叫做流输出.这个阶段让我们可以把gpu用作一个几何引擎.相对于把处理过的顶点传输到余下的管线,我们可以将它们输出到一个数组中以备做更多处理.这些数据可以被cpu或者gpu在之后的阶段中使用.这一阶段通常也用于模拟粒子.
#### 2.3.3 裁剪
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;只有全部或部分在观察体重的图元需要传输到光栅化阶段(以及接下来的像素处理阶段),然后将它们绘制在屏幕上.只有一部分在观察体中的图元需要被裁剪.例如,一条直线的一个顶点在观察体外,另一个在观察体内,这条直线将会被观察体裁剪,观察体外的顶点将被交接处的顶点替代.应用投影矩阵意味着转换过的图元被单位立方体裁剪.在裁剪之前应用观察变换和投影变换的好处是:这让裁剪问题保持一致,即图元总是按照单位立方体来裁剪.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;裁剪的过程如图2.6所示.除了观察体的六个裁剪平面之外,用户也可以定义额外的裁剪平面.![image.png](RTR.2.6.png)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;裁剪步骤使用4值的由投影生成的齐次坐标系来执行.三角形中的值通常不会在投影空间做线性插值.第四个坐标是需要的,以保证数据正确的被插值和裁剪当投影变换被应用的时候(???).最后执行透视除法,将三角形的位置信息映射到三维的归一化了的设备坐标系中.几何阶段的最后一步是转换到窗口坐标系.
#### 2.3.4 屏幕映射
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;只有观察体中的图元被传输到屏幕映射阶段,并且进入这个阶段时坐标系仍然是三维的.每个图元的x和y坐标轴都被转换到屏幕坐标系.屏幕坐标系加上z轴也叫窗体坐标系.假设场景被绘制到一个最小角为$(x_{1},y_{1})$,最大角为$(x_{2},y_{2})$的窗体上,那么屏幕映射就是一个缩放变换跟着一个平移变换.新的x轴和y轴组成屏幕坐标系.对于OpenGL来说$[-1,1]$也被映射到$[z_{1},z{2}]$(对DirectX来说是$[0,1]$),默认情况下$z_{1}=0$,$z_{2}=1$.这些值可以通过API来修改.窗体坐标系和被重新映射过的z值传输到光栅化阶段.屏幕映射的过程如图2.7所示.![image.png](RTR.2.7.png)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;给定一个笛卡尔坐标系下的水平像素数组,最左边的像素是0.0.像素的中心是0.5.于是$[0,9]$范围内的像素覆盖了$[0.0,10.0]$的区域.转换方程如下:
$$d=floor(c)$$ $$c=d+0.5$$ d是像素一个离散的索引,c是像素内连续的值.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;所有的API都有位置值并从左至右逐渐增长.对OpenGL和DirectX来说顶部和底部的零值在某些情况下不一致.OpenGL倾向于使用笛卡尔坐标系--以左下角为最小位置,但DirectX有时左上角为最小位置,要依据上下文来决定.两种表示方式都符合逻辑.
### 2.4 光栅化
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;给定经过变换和投影的顶点以及它们的着色数据(所有这些数据都来自于几何处理阶段),下一个阶段的目标是找出图元内的所有元素,这一阶段叫做光栅化,并且可分为两个阶段:三角形设置(也叫做图元装备)和三角形遍历.如图2.8所示![image.png](RTR.2.8.png)也可以处理点和线,但通常是三角形.光栅化也叫做扫描转换,即从屏幕空间的带有z值的二维顶点以及和顶点相关的多种着色信息转换成屏幕上的像素.光栅化也可以被当做几何处理阶段和像素处理阶段的同步点.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;三角形是否重叠取决于你怎样设置管线.